# 0. Setup

In [ ]:
# Mount to Google Drive
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision.models import vgg19
from PIL import Image
import numpy as np
import os

# TPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
folder_path = '/content/drive/MyDrive/SR-ATS'

import os
os.chdir(folder_path+'/functions')

# Test

In [ ]:
import net_tester
import data_loader

set14_path = data_loader.un_zip("set14")
set5_path  = data_loader.un_zip("set5")
bsd_path   = data_loader.un_zip("BSD100")
urban_path = data_loader.un_zip("urban100")

#net_tester.test_set(set14_path, device, model, save_path=None)


In [ ]:
import net
import importlib
importlib.reload(net)

# Load Model
model_name = "v3"
model = torch.load("../models/save/SR-ATS-3.pth")
model.eval()
model.to(device)

torch.save(model, f"../models/save/model_{model_name}.pth")

In [ ]:
print("[set5]")
net_tester.test_set(set5_path , device, model, save_path=f"../outputs/{model_name}/s5")

print("\n[set14]")
net_tester.test_set(set14_path, device, model, save_path=f"../outputs/{model_name}/s14")

print("\n[bsd]")
net_tester.test_set(bsd_path  , device, model, save_path=f"../outputs/{model_name}/bsd")

print("\n[urban]")
net_tester.test_set(urban_path, device, model, save_path=f"../outputs/{model_name}/urb")


# Test Images (Fig3)

In [ ]:
net_tester.test_set("../test/GT", device, model, save_path=f"../test/v3")

/content/drive/MyDrive/SR-ATS/functions/net_tester.py:16: UserWarning: Inputs have mismatched dtype.  Setting data_range based on image_true.
  return metrics.peak_signal_noise_ratio(img1, img2)


psnr: 27.79811691400675, ssim: 0.79680049160359


In [ ]:
from PIL import Image
import os

def process_images(input_folder, output_folder):
    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img = Image.open(os.path.join(input_folder, filename))
            width, height = img.size
            img_downsampled = img.resize((width // 4, height // 4), Image.BICUBIC)
            img_upscaled = img_downsampled.resize((width, height), Image.BICUBIC)
            output_path = os.path.join(output_folder, filename)
            img_upscaled.save(output_path)
            print(f"Processed and saved: {output_path}")

input_folder = '../test/GT'
output_folder = '../test/bicubic'

os.makedirs(output_folder, exist_ok=True)

process_images(input_folder, output_folder)


Processed and saved: ../test/bicubic/bsd040.png
Processed and saved: ../test/bicubic/img_019.png
Processed and saved: ../test/bicubic/img_021.png
Processed and saved: ../test/bicubic/img_028.png
Processed and saved: ../test/bicubic/img_067.png
Processed and saved: ../test/bicubic/img_100.png


# Visualization (Fig 4)

In [ ]:
print(model)

SR_v3(
  (vgg_3): relu_3_4(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)


In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import torch

image_path = "../outputs/visualization/bsd040.png"

image = Image.open(image_path)
transform = transforms.Compose([
    transforms.ToTensor(),
])
image_tensor = transform(image)

import torch.nn.functional as F
image_tensor = F.interpolate(image_tensor.unsqueeze(0), scale_factor=0.25, mode='bicubic')


>input

In [ ]:
import torch

layer_name = "inp-feat"

features = []
def get_features(module, input, output):
    features.append(input[0])

layer = model.d3.upsample
hook = layer.register_forward_hook(get_features)

output = model(image_tensor.to(device))

feature_map = features[0]

hook.remove()

In [ ]:
save_dir = f'../outputs/visualization/{layer_name}'
os.makedirs(save_dir, exist_ok=True)

batch_size, num_channels, height, width = feature_map.shape

for i in range(num_channels):
    feature = feature_map[0, i].detach().cpu().numpy()

    # Normalize to [0, 255] for visualization
    feature = (feature - feature.min()) / (feature.max() - feature.min())
    feature = (feature * 255).astype('uint8')

    img = Image.fromarray(feature)
    save_path = os.path.join(save_dir, f'{layer_name}_{i+1}.png')
    img.save(save_path)

> output

In [ ]:
import torch

layer_name = "out-feat"

features = []
def get_features(module, input, output):
    features.append(input[0])

layer = model.d2.upsample
hook = layer.register_forward_hook(get_features)

output = model(image_tensor.to(device))

feature_map = features[0]

hook.remove()

In [ ]:
save_dir = f'../outputs/visualization/{layer_name}'
os.makedirs(save_dir, exist_ok=True)

batch_size, num_channels, height, width = feature_map.shape
print(feature_map.shape)

for i in range(num_channels):
    feature = feature_map[0, i].detach().cpu().numpy()

    # Normalize to [0, 255] for visualization
    feature = (feature - feature.min()) / (feature.max() - feature.min())
    feature = (feature * 255).astype('uint8')

    img = Image.fromarray(feature)
    save_path = os.path.join(save_dir, f'{layer_name}_{i+1}.png')
    img.save(save_path)

torch.Size([1, 256, 120, 80])


> high freq

In [ ]:
import torch

layer_name = "high-freq"

features = []
def get_features(module, input, output):
    features.append(output)

layer = model.d3.c6
hook = layer.register_forward_hook(get_features)

output = model(image_tensor.to(device))

feature_map = features[0]

hook.remove()

save_dir = f'../outputs/visualization/{layer_name}'
os.makedirs(save_dir, exist_ok=True)

batch_size, num_channels, height, width = feature_map.shape

for i in range(num_channels):
    feature = feature_map[0, i].detach().cpu().numpy()

    # Normalize to [0, 255] for visualization
    feature = (feature - feature.min()) / (feature.max() - feature.min())
    feature = (feature * 255).astype('uint8')

    img = Image.fromarray(feature)
    save_path = os.path.join(save_dir, f'{layer_name}_{i+1}.png')
    img.save(save_path)

> low frequency

In [ ]:
# low-freq
layer_name = "low-freq"

vgg3 = model.vgg_3.to(device)

LR4x = F.interpolate(image_tensor, scale_factor=4, mode='bicubic')

f = vgg3(LR4x.to(device))

batch_size, num_channels, height, width = f.shape

save_dir = f'../outputs/visualization/{layer_name}'
os.makedirs(save_dir, exist_ok=True)

batch_size, num_channels, height, width = feature_map.shape

for i in range(num_channels):
    feature = f[0, i].detach().cpu().numpy()

    # Normalize to [0, 255] for visualization
    feature = (feature - feature.min()) / (feature.max() - feature.min())
    feature = (feature * 255).astype('uint8')

    img = Image.fromarray(feature)
    save_path = os.path.join(save_dir, f'{layer_name}_{i+11}.png')
    img.save(save_path)